# TikTok imports

In [2]:
import pandas as pd
import time
from tqdm.auto import tqdm
! pip install requests
import requests
from datetime import datetime

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install ensembledata
from ensembledata.api import EDClient


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# initialize the ensembledata client
client = EDClient("2BJVoG42lL9XLjlU")

# get a free token at https://dashboard.ensembledata.com/register
# free trial of 50 units per day for 7 days, starting 23/11/24
#
# costs:
#  - 1 unit = 200 posts in full_hashtag_search :)
#  - 1 unit = 20 posts in hashtag_search
#  - 1 unit = 20 posts in keyword_search or full_keyword_search :(
#  - 1 unit = 30 comments to a post
#  - 1 unit = 30 replies to a comment
#
# documentation available at https://ensembledata.com/apis/docs#tag/Tiktok
# and https://github.com/EnsembleData/tiktok-scraper
# source code available at
# https://github.com/EnsembleData/ensembledata-python/tree/main/ensembledata/api

# Read posts from TikTok

Hashtag search (200 posts at the cost of 1 unit)

In [5]:
# read posts related to an hashtag
result = client.tiktok.full_hashtag_search(
                hashtag = "rome", # hashtag name
                days = 360, # filter out posts created more than N days ago
                remap_output = True, # False for mobile app, True for web
                max_cursor = 200) # max number of posts fetched,
                                  # cost is 1 unit per 200 posts

In [6]:
# convert json to dataframe
df = pd.json_normalize(result.data["posts"])

# extract relevant info
df = df[['itemInfos.id', 'itemInfos.createTime', 'itemInfos.authorId',
         'itemInfos.text', 'itemInfos.video.urls', 'itemInfos.shareCount',
         'itemInfos.playCount', 'itemInfos.commentCount']]

# rename columns
df.columns = ['id', 'createTime', 'authorId', 'text', 'video.urls',
              'shareCount', 'playCount', 'commentCount']

# format time to date in string format
df["createTime"] = [str(datetime.fromtimestamp(int(i)).date())
                                                for i in df["createTime"]]

# save
df_hashtag = df
df_hashtag.to_csv('rome__tiktok_hashtag_posts.csv', index=False)

In [7]:
display(df_hashtag)

,id,createTime,authorId,text,video.urls,shareCount,playCount,commentCount
0,7380487798517697809,2024-06-15,7372729660905751553,Things to do in Rome 🍝✨ #rome #italy #travel #...,[https://v45.tiktokcdn-eu.com/8ffb8f6821650b28...,1033,138499,79
1,7394176466340089121,2024-07-21,6837200889528206342,"When it’s hot outside, underground is always a...",[https://v45.tiktokcdn-eu.com/04b70ce539dbcb21...,4422,483706,88
2,7415725536007195910,2024-09-18,6790121256747648006,Being a tourist 😂 #rome #fyp #italy #italytrav...,[https://v45.tiktokcdn-eu.com/e481fe08cbf6781b...,144,59773,16
3,7425195042567884038,2024-10-13,7146068440930386949,Maybe Rome is currently not the best option 🥲 ...,[https://v45.tiktokcdn-eu.com/2971af24341ad5e7...,7961,1028794,143
4,7358144332630625568,2024-04-15,7148038032389489670,Where to go out in Rome Italy? If you're looki...,[https://v45.tiktokcdn-eu.com/21b05ff3b1f24226...,1980,180946,44
...,...,...,...,...,...,...,...,...
123,7441157781836647712,2024-11-25,7280469526700540961,ROME - Italy 📍🇮🇹. #italy ...,[https://v16m.tiktokcdn-eu.com/5159d822e46f3cd...,0,796,2
124,7441276169166572833,2024-11-25,6778524591850701829,pov: you went to the famous mirror in rome #ro...,[https://v16m.tiktokcdn-eu.com/8611cbf7ea068b6...,0,341,0
125,7360789966743555333,2024-04-22,6728727889942922246,living in rome 💔 #rome #roma #romeitaly #italy...,[https://v16m.tiktokcdn-eu.com/373bcf0899d6d98...,10,2975,0
126,7378485905440476449,2024-06-09,7211418901115094021,When in Rome… ❤️ #rome#visitrome#colosseo#roma...,[https://v16m.tiktokcdn-eu.com/827f08e48034ee3...,4,2470,5


Keyword search (100 posts at the cost of 5 units) - limited to 180 days!

In [8]:
# read posts related to a keyword
keyword = "visit rome" # keyword name
period = 180 # must be one of 0, 1, 7, 30, 90, 180 days
             # returns posts more recent than the given period
country = "us" # leave blank "" for any country
max_cursor = 100 # max number of posts fetched, cost is 1 unit per 20 posts

# loop through keyword_search to control for max_cursor
posts = list()
cursor = None
for i in tqdm(range(int(max_cursor/20))):
    result = client.tiktok.keyword_search(keyword = keyword,
                                          period = period,
                                          country = country,
                                          cursor = cursor)
    posts.extend(result.data["data"])
    cursor = result.data.get("nextCursor")

    # If there is no next cursor, we've fetched all the available posts
    if cursor is None:
        break

100%|██████████| 5/5 [00:40<00:00,  8.04s/it]


In [ ]:
# convert json to dataframe
df = pd.json_normalize(posts)

# extract relevant info
df = df[['aweme_info.aweme_id', 'aweme_info.create_time',
         'aweme_info.author.uid', 'aweme_info.desc',
         'aweme_info.video.play_addr.url_list',
         'aweme_info.statistics.share_count',
         'aweme_info.statistics.play_count',
         'aweme_info.statistics.comment_count']]

# rename columns
df.columns = ['id', 'createTime', 'authorId', 'text', 'video.urls',
              'shareCount', 'playCount', 'commentCount']

# format time to date in string format
df["createTime"] = [str(datetime.fromtimestamp(int(i)).date())
                                                for i in df["createTime"]]

# save
df_keyword = df
df_keyword.to_csv('rome_tiktok_keyword_posts.csv', index=False)

In [ ]:
display(df_keyword)

# Read comments from TikTok

Comments search

In [ ]:
df_hashtag = pd.read_csv('rome_tiktok_hashtag_posts.csv')
df_keyword = pd.read_csv('rome_tiktok_keyword_posts.csv')

# order by comment count
df_hashtag = df_hashtag.sort_values(by=['commentCount'], ascending=False).reset_index(drop=True)
df_keyword = df_keyword.sort_values(by=['commentCount'], ascending=False).reset_index(drop=True)
# delete posts with less than 30 comments
df_hashtag = df_hashtag[df_hashtag['commentCount']>=30]
df_keyword = df_keyword[df_keyword['commentCount']>=30]
# state how many
print(f'{len(df_hashtag)} posts for hastags with more than 30 comments')
print(f'{len(df_keyword)} posts for keywords with more than 30 comments')

In [ ]:
display(df_hashtag)

In [ ]:
# set search criteria
comments_to_retrieve = 60 # 30 is a safe number = 1 credit!

comments = list()
for i in range(2): # change the range to get more!
    aweme_id = df_hashtag["id"][i]
    cursor = None
    for _ in range(int(comments_to_retrieve/30)):
        result = client.tiktok.post_comments(
            aweme_id = aweme_id,
            cursor = cursor
        )
        comments.extend(result.data["comments"])
        cursor = result.data.get("nextCursor")
        if cursor is None:
            break

In [ ]:
# convert json to dataframe
df = pd.json_normalize(comments)

# extract relevant info
df = df[['aweme_id', 'cid', 'create_time', 'text', 'reply_comment_total']]

# format time to date in string format
df["create_time"] = [str(datetime.fromtimestamp(int(i)).date())
                                                for i in df["create_time"]]

# order by number of replies
df = df.sort_values(by=['reply_comment_total'], ascending=False).reset_index(drop=True)

# save
df_comments = df
df_comments.to_csv('rome_tiktok_hashtag_comments.csv', index=False)

In [ ]:
display(df_comments)

Replies search

In [ ]:
# set search criteria
replies_to_retrieve = 30 # 30 is a safe number = 1 credit!

replies = list()
for i in range(4): # change the range to get more!
    aweme_id = df_comments["aweme_id"][i]
    cid = df_comments["cid"][i]
    cursor = None
    for _ in range(int(replies_to_retrieve/30)):
        result = client.tiktok.post_comment_replies(
            aweme_id = aweme_id,
            comment_id = cid,
            cursor = cursor
        )
        replies.extend(result.data["comments"])
        cursor = result.data.get("nextCursor")
        if cursor is None:
            break

In [ ]:
# convert json to dataframe
df = pd.json_normalize(replies)

# extract relevant info
df = df[['aweme_id', 'cid', 'reply_id', 'create_time', 'text']]

# format time to date in string format
df["create_time"] = [str(datetime.fromtimestamp(int(i)).date())
                                                for i in df["create_time"]]

# save
df_replies = df
df_replies.to_csv('rome_tiktok_hashtag_replies.csv', index=False)

In [ ]:
display(df_replies)